# 优化算法

## 1. 微批梯度下降

批量梯度下降一次性将所有样本进行矩阵运算，这个向量化运算的过程，利用CPU或GPU的并行计算指令集，可以获得比单个样本计算更高的效率。

当样本量过大时，比如来到百万级或以上，视内存或显存的限制而定，对超大的矩阵进行运算，反而会导致效率下降。微批梯度下降的思路，就是将训练样本分组，对每个组（每个批次）进行梯度下降运算。

对每个批次的一次前向传播和反向传播计算过程，称为一个**阶段 epoch**。而将所有批次进行计算完成后，参数还不会收敛。需要持续循环。

![Mini-batch gradient descent](img/Mini-batch gradient descent.png)

## 2. 理解微批梯度下降

批量梯度下降的成本函数在每次迭代后是严格下降的，而对于微批梯度下降则不一定。只要学习速率合适，微批梯度下降的成本函数总体会呈现下降趋势。但由于每个批次中存在的噪声，有时成本函数也会上升。

![Training with mini batch gradient descent](img/Training with mini batch gradient descent.png)

如果批次大小等于样本大小，微批梯度下降就是批量梯度下降，批量梯度下降的每一次迭代需要很长的时间；如果批次大小等于1，这种情况下的微批梯度下降就是随机梯度下降，随机梯度下降损失了向量化提升效率的部分；微批梯度下降的每个批次都利用了向量化，同时每次批次的计算会比训练集全量计算快。

![Choosing your mini-batch size](img/Choosing your mini-batch size.png)

当样本量较小（m < 2000）时，建议使用批量梯度下降。使用微批梯度下降的典型批次是64,128,256,512。它们都是2的整数次幂，对于内存/显存来说更为友好。至于究竟是选用64还是512，则取决于单个批次能否在内存/显存中存储。

![Choosing your mini-batch size2](img/Choosing your mini-batch size2.png)

## 3. 指数加权移动平均值

指数加权移动平均值可以使数据的变化更为平缓。
$$ 
\begin{split}
V_0 &= 0 \\
V_t &= \beta V_{t-1} + (1-\beta)\theta_t \\
\end{split}
$$
上面这个变换之后的 $V_t$，可以看做是针对过去 $\frac{1}{1-\beta}$ 个单位 $\theta$ 的平均值。这样，$\beta$ 越接近 $1$，数据的变化就越平缓，曲线会向右偏（平均了更多天的数据）。

![Exponentially weighted averages](img/Exponentially weighted averages.png)

## 4. 理解指数加权移动平均值

考虑偏差修正（bias correction）之后，$V_t$ 实际上可以看做是 $\theta_1, \theta_2, ..., \theta_t$ 的指数加权。

另外由于 $\lim_{\epsilon \rightarrow 1}(1-\epsilon)^{\frac{1}{\epsilon}} = \frac{1}{e}$，所以令 $\beta = 1 - \epsilon$，$\beta^{\frac{1}{1-\beta}} \approx \frac{1}{e}$。

![Exponentially weighted averages 2](img\Exponentially weighted averages 2.png)

指数加权移动平均值，相比真正的平均值的一个优点，是其在实现的细节中，它可以写成迭代过程，每轮迭代只需要当前的 $\theta_t$ 和上一轮迭代结果 $V_{t-1}$，占用的内存很少。同时它只是对均值的近似，从精度上并不能完美替代保留整个移动窗口计算平均值。由于节省内存以及相对不错的预测精度，指数加权移动平均值在机器学习中的应用非常广泛。

![Implementing exponentially weighted averages](img\Implementing exponentially weighted averages.png)

## 5. 偏差修正

按照上述方法计算的指数加权移动平均值，并不是严格的指数加权。尤其因为 $V_0=0$，初始值的偏差会比较大。解决的方法是将公式修正为
$$ \frac{V_t}{1-\beta^t} = \beta V_{t-1} + (1-\beta)\theta_t $$

在机器学习的算法实现中，事实上很少有人会去做偏差修正。因为随着时间推移，后续的偏差值会越来越小。

![Bias correction](img/Bias correction.png)

## 6. 带有动量的梯度下降

一句话概括带有动量的梯度下降算法：计算梯度的指数加权移动平均，通过这个值进行梯度更新。带有动量的梯度下降算法几乎总是比原版的梯度下降更快。带有动量的梯度下降算法会减小梯度下降过程中的震荡。

![Gradient descent example](img/Gradient descent example.png)

在某些机器学习的文献中会忽略 $1-\beta$ 这一项。$\beta$ 的值通常会固定为0.9，当然也可以当做超参来调节。

![Momentum implementation details](img/Momentum implementation details.png)

## 7. RMSprop

RMS的意思是**平方平均数 root mean square**。

![RMSprop](img/RMSprop.png)

## 8. Adam

机器学习发展的历程中，学者们发明过各种各样的优化算法，但绝大部分的优化算法都只能解决小一块领域的问题，不能做到普遍适用。很多研究人员都倾向于认为带有动量的梯度下降已经足够好了。而RMSprop和Adam可以说是优化算法中少有的，在深度学习中广泛适用的优化算法。

**Adam(Adaptive moment estimation)**可以看做是一种结合了带有动量的梯度下降，和RMSprop的优化算法。其中适用动量梯度下降的过程中，会用到偏差修正。

![Adam optimization algorithm](img/Adam optimization algorithm.png)

Adam算法的超参当中，通常需要调的只有学习速率 $\alpha$，$\beta_1$ 和 $\beta_2$ 可以调但更多情况下使用默认值就好，$\epsilon$ 对结果几乎没有影响，但Adam论文的作者建议使用 $10^{-8}$。

![Adam Hyperparameters choice](img/Adam Hyperparameters choice.png)